In [ ]:
from aymurai.utils.json_data import load_json, save_json, group_by_key

## Load data

In [ ]:
# load data
data = load_json("/resources/annotations/label-studio/resos-annotations/30-nov/project-3-at-2022-11-30-16-04-2b43bf39.json")
len(data)

In [ ]:
# pick one sample
sample = data[-1]

In [ ]:
sample.keys()

In [ ]:
sample["annotations"]

In [ ]:
len(sample["annotations"])

In [ ]:
len(sample["annotations"][0])

In [ ]:
len(sample["annotations"][0]["result"])

In [ ]:
sample["annotations"][0]["result"][-10:]

## No decision

In [ ]:
def filter_decision(annot: dict) -> bool:
    try:
        labels = annot["value"]["labels"]
        if "DECISION" not in labels:
            return True
        
    except:
        try:
            from_name = annot["from_name"]
            if from_name != "DECISION":
                return True
        
        except:
            return False

In [ ]:
no_decision = list(filter(filter_decision, sample["annotations"][0]["result"]))
len(no_decision)

In [ ]:
for sample in data:
    no_decision = list(filter(filter_decision, sample["annotations"][0]["result"]))
    sample["annotations"][0]["result"] = no_decision

In [ ]:
len(data[-1]["annotations"][0]["result"])

In [ ]:
file_path = "/resources/annotations/label-studio/resos-annotations/30-nov/no_decision.json"
save_json(data, file_path)

## Grouped by key

In [ ]:
grouped = list(group_by_key(sample["annotations"][0]["result"], group_key="id", sort_key="id"))

In [ ]:
len(grouped)

In [ ]:
[len(group) for group in grouped]

In [ ]:
grouped[-1]

## Format for extracting subcategories

In [ ]:
from more_itertools import collapse

In [ ]:
list(collapse(grouped[-1]))

In [ ]:
outputDict1 = dict((k, set()) for k in collapse(grouped[-1]))

for d in grouped[-1]:
    keys = d.keys()
    values = d.values()
    for k, v in zip(keys, values):
        if not isinstance(v, dict):
            outputDict1[k].add(v)
        else:
            if not isinstance(outputDict1[k], dict):
                outputDict1[k] = dict()
            outputDict1[k] = outputDict1[k] | v

outputDict1

In [ ]:
grouped[-1]

In [ ]:
from copy import deepcopy
from typing import Union, Any


def parse_set(set_: set) -> Union[Any, list]:
    list_ = list(set_)
    if len(list_) == 1:
        return list_[0]
    return list_

In [ ]:
def parse_annotations(annots: list[dict]) -> dict:
    output_dict = dict((k, set()) for k in collapse(grouped[-2]))

    for d in annots:
        keys = d.keys()
        values = d.values()

        for k, v in zip(keys, values):
            if not isinstance(v, dict):
                output_dict[k].add(v)

            else:
                if not isinstance(output_dict[k], dict):
                    output_dict[k] = dict()
                output_dict[k] = output_dict[k] | v

    output_dict = output_dict["value"] | {"id": output_dict["id"]}

    output_dict = {
        k: parse_set(v) if k not in ("labels", "choices") and isinstance(v, set) else v for k, v in output_dict.items()
    }


    return output_dict


In [ ]:
parse_annotations(grouped[-1])

In [ ]:
parse_annotations(grouped[-2])

In [ ]:
annotations = [parse_annotations(group) for group in grouped]

In [ ]:
len(annotations)

In [ ]:
annotations

In [ ]:
annotations[0].keys()

In [ ]:
print(len(annotations))
filtered_annotations = [
    annotation
    for annotation in annotations
    if "labels" in list(annotation.keys()) and annotation["labels"][0] != "DECISION"
]
print(len(filtered_annotations))
